# Tutorial: Private, Decentralized Deep Learning

Welcome to PySyft's introductory tutorial for privacy preserving, decentralized deep learning. This series of notebooks is a step-by-step guide for you to get to know the new tools and techniques required for doing deep learning on secret/private data/models without centralizing them under one authority.

**Scope:** Note that we'll not just be talking about how to decentralize / encrypt data, but we'll be addressing how PySyft can be used to help decentralize the entire ecosystem around data, even including the Databases where data is stored and queried, and the neural models which are used to extract information from data. As new extensions to PySyft are created, these notebooks will be extended with new tutorials to explain the new functionality.

Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

## Outline:

- Part 1: The Basic Tools of Private, Decentralized Data Science
- Part 2: Intro to Federated Learning
- Part 3: Advanced Remote Execution Tools
- Part 4: Federated Learning via Trusted Aggregator
- Part 5: Intro to Encrypted Programs
- Part 6: Build an Encrypted, Decentralized Database
- Part 7: Build an Encrypted, Decentralized Ledger
- Part 8: Train an Encrypted Neural Network on Encrypted Data


## Why Take This Tutorial?

**1) A Competitive Career Advantage** - For the past 20 years, the digital revolution has made data more and more accessible in ever larger quantities as analog processes have become digitized. However, with new regulation such as [GDPR](https://eugdpr.org/), enterprises are under pressure to have less freedom with how they use - and more importantly how they analyze - personal information. **Bottom Line:** Data Scientists aren't going to have access to as much data with "old school" tools, but by learning the tools of Private Deep Learning, YOU can be ahead of this curve and have a competitive advantage in your career. 

**2) Entrepreneurial Opportunities** - There are a whole host of problems in society that Deep Learning can solve, but many of the most important haven't been explored because it would require access to incredibly sensitive information about people (consider using Deep Learning to help people with mental or relationship issues!). Thus, learning Private Deep Learning unlocks a whole host of new startup opportunities for you which were not previously available to others without these toolsets.

**3) Social Good** - Deep Learning can be used to solve a wide variety of problems in the real world, but Deep Learning on *personal information* is Deep Learning about people, *for people*. Learning how to do Deep Learning on data you don't own represents more than a career or entrepreneurial opportunity, it is the opportunity to help solve some of the most personal and important problems in people's lives - and to do it at scale.

## How do I get extra credit?

- Star PySyft on Github! - [https://github.com/OpenMined/PySyft](https://github.com/OpenMined/PySyft)
- Make a Youtube video teaching this notebook!


... ok ... let's do this!

# Part -1: Prerequisites

- Know PyTorch - if not then take the http://fast.ai course and come back
- Read the PySyft Framework Paper https://arxiv.org/pdf/1811.04017.pdf! This will give you a thorough background on how PySyft is constructed which will help things make more sense.

# Part 0: Setup

To begin, you'll need to make sure you have the right things installed. To do so, head on over to PySyft's readme and follow the setup instructions. TLDR for most folks is.

- Install Python 3.5 or higher
- Install PyTorch 0.3.1 (it MUST be this version)
- Clone PySyft (git clone https://github.com/OpenMined/PySyft.git)
- cd PySyft
- pip install -r requirements.txt
- python setup.py install
- python setup.py test

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
# Run this cell to see if things work
import syft as sy
hook = sy.TorchHook() # always run this when you import syft

sy.FloatTensor([1,2,3,4,5])


 1
 2
 3
 4
 5
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

If this cell executed, then you're off to the races! Let's do this!

# Part 1: The Basic Tools of Private, Decentralized Data Science

So - the first question you may be wondering is - How in the world do we train a model on data we don't have access to? 

Well, the answer is surprisingly simple. If you're used to working in PyTorch, then you're used to working with torch.Tensor objects like these!

In [2]:
import torch
x = torch.FloatTensor([1,2,3,4,5])
y = x + x
print(y[0])

2.0


Obviously, using these super fancy (and powerful!) tensors is important, but also requires you to have the data on your local machine. This is where our journey begins. 

# Section 1.1 - Sending Tensors to Bob's Machine

Whereas normally we would perform data science / deep learning on the machine which holds the data, now we want to perform this kind of computation on some OTHER machine. More specifically, we can no longer simply assume that the data is on our local machine.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [3]:
bob = sy.VirtualWorker(id="bob")

For all intents and purposes, Bob's machine is on another planet - perhaps on Mars! But, at the moment the machine is empty. Let's create some data so that we can send it to Bob and learn about pointers!

In [4]:
x = torch.FloatTensor([1,2,3,4,5])
y = torch.FloatTensor([1,1,1,1,1])

And now - let's send our tensors to Bob!!

In [5]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

BOOM! Now Bob has two tensors! Don't believe me? Have a look for youself!

In [6]:
bob._objects

{26311085192: [_LocalTensor - id:26311085192 owner:bob],
 81620284840: [_LocalTensor - id:81620284840 owner:bob]}

Now notice something. When we called "x.send(bob)" it returned a new object "x_ptr". This is our first *pointer* to a tensor. Pointers to tensors do NOT actually hold data themselves. Instead, they simply contain metadata about a tensor (with data) stored on another machine. The purpose of these tensors is to give us an intuitive API to tell the other machine to compute functions using this tensor. Let's take a look at the metadata pointers contain.

In [7]:
x_ptr

FloatTensor[_PointerTensor - id:49511658927 owner:me loc:bob id@loc:26311085192]

Check out that metadata! First, ID of our tensor is 49511658927, that makes sense... it was allocated a random ID (so if you're running this notebook the tensor probably has a different ID). However, there's also a few other pieces of metadata.


- loc: bob
- id@loc: 26311085192
- owner: me

Hopefully the naming of these attributes is quite intuitive. "loc" is short for "location" and it is a reference to the location that the pointer is pointing to! See?

In [8]:
x_ptr.location

<syft.core.workers.virtual.VirtualWorker id:bob>

In [9]:
bob

<syft.core.workers.virtual.VirtualWorker id:bob>

In [10]:
bob == x_ptr.location

True

The "id@loc" parameter is similar. It tells us the id that the Tensor object on Bob's machine has (the one that we're pointing to). See?

In [11]:
x_ptr.id_at_location

26311085192

In [12]:
bob._objects[x_ptr.id_at_location].parent


 1
 2
 3
 4
 5
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

And finally - we have the third attribute "owner: me" which is very similar to ".location". However, instead of specifying where the pointer is pointing, it specifies the owner of the pointer itself, which is me. 

Fun fact, just like we had a VirtualWorker object for Bob, we (by default) always have one for us as well. This worker is automatically created when we called "hook = sy.TorchHook()" and so you don't usually have to create it yourself.

In [13]:
me = sy.local_worker
me

<syft.core.workers.virtual.VirtualWorker id:me>

In [14]:
x_ptr.owner

<syft.core.workers.virtual.VirtualWorker id:me>

In [15]:
me == x_ptr.owner

True

And finally, just like we can call .send() on a tensor, we can call .get() on a pointer to a tensor to get it back!!!

In [16]:
x_ptr

FloatTensor[_PointerTensor - id:49511658927 owner:me loc:bob id@loc:26311085192]

In [17]:
x_ptr.get()


 1
 2
 3
 4
 5
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [18]:
y_ptr

FloatTensor[_PointerTensor - id:90717780790 owner:me loc:bob id@loc:81620284840]

In [19]:
y_ptr.get()


 1
 1
 1
 1
 1
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [20]:
bob._objects

{}

And as you can see... Bob no longer has the tensors anymore!!! They've moved back to our machine!

# Section 1.2 - Using Tensor Pointers

So, sending and receiving tensors from Bob is great, but this is hardly Deep Learning! We want to be able to perform tensor _operations_ on remote tensors. Fortunately, tensor pointers make this quite easy!! You can just use poiners like you would normal tensors!

In [21]:
x = sy.FloatTensor([1,2,3,4,5]).send(bob)
y = sy.FloatTensor([1,1,1,1,1]).send(bob)

In [22]:
z = x + y

In [23]:
z

FloatTensor[_PointerTensor - id:36494558106 owner:me loc:bob id@loc:36165772126]

And voila! 

Behind the scenes, something very powerful happened. Instead of x and y computing an addition locally, a JSON command was serialized and sent to Bob, who performed the computation, created a tensor z, and then returned the pointer to z back to us!

If we call .get() on the pointer, we will then receive the result back to our machine!

In [24]:
z.get()


 2
 3
 4
 5
 6
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

### Torch Functions

This API has been extended to all of Torch's operations!!!

In [25]:
z = torch.add(x,y)
z

FloatTensor[_PointerTensor - id:88188883439 owner:me loc:bob id@loc:75171518237]

In [26]:
z.get()


 2
 3
 4
 5
 6
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

### Variables (including backpropagation!)

In [27]:
x = sy.Variable(sy.FloatTensor([1,2,3,4,5]), requires_grad=True).send(bob)
y = sy.Variable(sy.FloatTensor([1,1,1,1,1]), requires_grad=True).send(bob)

In [28]:
z = (x + y).sum()

In [29]:
z.backward()

In [30]:
x = x.get()

In [31]:
x

Variable containing:
 1
 2
 3
 4
 5
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [32]:
x.grad

Variable containing:
 1
 1
 1
 1
 1
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

So as you can see, the API is really quite flexible and capable of performing nearly any operation you would normaly perform in Torch on *remote data*. This lays the groundwork for our more advanced privacy preserving protocols such as Federated Learing, Secure Multi-Party Computation, and Differential Privacy

# Bonus Section - Common Errors


### Try to run operation between tensors which aren't on the same machine

If you try to do an operation between two tensors which aren't on the same machine, you'll get an error that looks like this!!!

In [33]:
x = sy.FloatTensor([1,2,3,4,5]).send(bob)
y = sy.FloatTensor([1,1,1,1,1])

In [34]:
z = y + x

Exception: All tensors must be on the same machine. 

You just tried to call an operation between a tensor on your local machine and a tensor on remote worker 'bob'. Call .send("bob") on the local tensor (the one with ID:77492290714) or .get() on the remote tensor (the one with ID:52324553865)

### Try to interact with pointers to tensors on a worker which no longer exist


Or, alternatively, if you try to interact with pointers to tensors on a worker which no longer exist, you'll see an error like this!!!

In [35]:
x = sy.FloatTensor([1,2,3,4,5]).send(bob)

# delete all objects on bob
bob._objects = {}

y = x + x

Exception: Tensor "90533353749" not found on worker "bob"!!!

You just tried to interact with an object ID:90533353749 on worker bob which does not exist!!! Use .send() and .get() on all your tensors to make sure they'reon the same machines.

If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object's pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven't already called .get() on this pointer!!!

### Forget to create a TorchHook object

If you try to send a tensor when torch wasn't hooked (you didn't created a TorchHook object), you'll see an error like this!!!

In [ ]:
# Restart the runtime, since in the current runtime you did create a TorchHook object
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [1]:
import syft as sy
# always run `hook = sy.TorchHook()` when you import syft, but let's say you forgot to :(
import torch

# Try to create a new tensor and send it to a worker
a = torch.FloatTensor([1,2,3,4,5])
a_ptr = a.send(bob)

AttributeError: 'torch.FloatTensor' object has no attribute 'send'

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on Github

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft Github Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for github issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)